In [ ]:
import json
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# 이미지 및 클래스 설정
LABEL_ROOT = "train/outputs_json"
IMAGE_ROOT = "data/train/DCM"
selected_classes = ['Trapezium', 'Trapezoid', 'Capitate', 'Hamate', 'Scaphoid', 'Lunate', 'Triquetrum', 'Pisiform']
CLASSES = [
    'finger-1', 'finger-2', 'finger-3', 'finger-4', 'finger-5',
    'finger-6', 'finger-7', 'finger-8', 'finger-9', 'finger-10',
    'finger-11', 'finger-12', 'finger-13', 'finger-14', 'finger-15',
    'finger-16', 'finger-17', 'finger-18', 'finger-19', 'Trapezium',
    'Trapezoid', 'Capitate', 'Hamate', 'Scaphoid', 'Lunate',
    'Triquetrum', 'Pisiform', 'Radius', 'Ulna',
]

# LABEL_ROOT 폴더의 모든 JSON 파일을 탐색하고 이미지별로 min, max 구하기
image_json_paths = {}
for root, _, files in os.walk(LABEL_ROOT):
    for file in files:
        if file.endswith(".json"):
            image_id = file.split(".")[0]
            if image_id not in image_json_paths:
                image_json_paths[image_id] = []
            image_json_paths[image_id].append(os.path.join(root, file))

# IMAGE_ROOT 폴더의 모든 PNG 파일을 탐색
image_paths = {}
for root, _, files in os.walk(IMAGE_ROOT):
    for file in files:
        if file.endswith(".png"):
            image_id = file.split(".")[0]
            image_paths[image_id] = os.path.join(root, file)


In [7]:

import json
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# 이미지 및 클래스 설정
LABEL_ROOT = "train/outputs_json"
IMAGE_ROOT = "train/DCM"
selected_classes = ['Trapezium', 'Trapezoid', 'Capitate', 'Hamate', 'Scaphoid', 'Lunate', 'Triquetrum', 'Pisiform']

# 저장 경로
CROPPED_IMAGE_DIR = "cropped_images"
CROPPED_ANNOTATION_DIR = "cropped_json"


# JSON 파일에서 클래스별 좌표 불러오기
def load_annotations(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data.get('annotations', [])

# 각 이미지에 대한 segmentation 값의 min, max 구하기
def get_image_segmentation_min_max(image_json_paths):
    all_x = []
    all_y = []

    for json_path in image_json_paths:
        annotations = load_annotations(json_path)

        for annotation in annotations:
            points = annotation['points']
            # 'Trapezium', 'Trapezoid', 'Capitate', 'Hamate', 'Scaphoid', 'Lunate', 'Triquetrum', 'Pisiform' 클래스에 해당하는 경우
            if annotation['label'] in selected_classes:
                x_coords = [point[0] for point in points]
                y_coords = [point[1] for point in points]
                all_x.extend(x_coords)
                all_y.extend(y_coords)

    # 전체 x, y 좌표의 최소값과 최대값 계산
    x_min, x_max = min(all_x), max(all_x)
    y_min, y_max = min(all_y), max(all_y)

    return x_min, x_max, y_min, y_max

# LABEL_ROOT 폴더의 모든 JSON 파일을 탐색하고 이미지별로 min, max 구하기
image_json_paths = {}
for root, _, files in os.walk(LABEL_ROOT):
    for file in files:
        if file.endswith(".json"):
            image_id = file.split(".")[0]  # 이미지 ID 추출 (예: ID001)
            if image_id not in image_json_paths:
                image_json_paths[image_id] = []
            image_json_paths[image_id].append(os.path.join(root, file))

# IMAGE_ROOT 폴더의 모든 PNG 파일을 탐색
image_paths = {}
for root, _, files in os.walk(IMAGE_ROOT):
    for file in files:
        if file.endswith(".png"):
            image_id = file.split(".")[0]
            image_paths[image_id] = os.path.join(root, file)

# 이미지 별 min, max 값을 계산
image_min_max = {}
for image_id, json_paths in image_json_paths.items():
    x_min, x_max, y_min, y_max = get_image_segmentation_min_max(json_paths)
    image_min_max[image_id] = {
        'x_min': x_min,
        'x_max': x_max,
        'y_min': y_min,
        'y_max': y_max
    }

# 이미지 ID 선택
for image_id, image_path in image_paths.items():
    if image_id not in image_min_max:
        continue

    # 해당 이미지에 대한 min, max 값을 가져오기
    x_min, x_max = image_min_max[image_id]['x_min'], image_min_max[image_id]['x_max']
    y_min, y_max = image_min_max[image_id]['y_min'], image_min_max[image_id]['y_max']

    # 이미지 로드
    image = cv2.imread(image_path)

    # 512x512 사이즈로 크롭 범위 설정
    crop_width = 512
    crop_height = 512

    # 크롭 범위 확장
    crop_x_start = max(x_min - (crop_width - (x_max - x_min)) // 2, 0)
    crop_x_end = crop_x_start + crop_width
    crop_y_start = max(y_min - (crop_height - (y_max - y_min)) // 2, 0)
    crop_y_end = crop_y_start + crop_height

    # 이미지 크기 (경계를 벗어나지 않도록 조정)
    crop_x_end = min(crop_x_end, image.shape[1])  # x 최대값을 이미지 너비로 제한
    crop_y_end = min(crop_y_end, image.shape[0])  # y 최대값을 이미지 높이로 제한

    # 크롭된 영역 가져오기
    cropped_image = image[crop_y_start:crop_y_end, crop_x_start:crop_x_end]

    # point 불러오기
    annotations = []
    for json_path in image_json_paths[image_id]:
        annotations.extend(load_annotations(json_path))

    # point 변환 및 저장
    cropped_annotations = {}
    cropped_annotations['annotations']=[]
    for json_path in image_json_paths[image_id]:
        annotations = load_annotations(json_path)
        for annotation in annotations:
            if annotation['label'] in CLASSES:
                adjusted_points = [
                    [point[0] - crop_x_start, point[1] - crop_y_start]
                    for point in annotation['points']
                        if crop_x_start <= point[0] < crop_x_end and crop_y_start <= point[1] < crop_y_end
                    ]
                if adjusted_points:
                    cropped_annotations['annotations'].append({ 
                        'label': annotation['label'],
                        'points': adjusted_points
                    })
    #저장
    imageroot = image_path.split("/"+image_id)[0]
    os.makedirs(CROPPED_IMAGE_DIR+"/"+imageroot, exist_ok=True)

    jsonroot = json_path.split("/"+image_id)[0]
    os.makedirs(CROPPED_ANNOTATION_DIR+"/"+jsonroot, exist_ok=True)

    cropped_image_path = os.path.join(CROPPED_IMAGE_DIR,image_path)
    cv2.imwrite(cropped_image_path, cropped_image)

    cropped_annotation_path = os.path.join(CROPPED_ANNOTATION_DIR,json_path)
    with open(cropped_annotation_path, 'w') as f:
        json.dump(cropped_annotations, f)



In [3]:
image_path

'train/DCM/ID311/image1664241629103.png'